In [1]:
import pandas as pd
import numpy as np
import os
import sys
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pyarrow.parquet as pq
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score
import xgboost as xgb

Версия с эмбеддингами. Кэтбуст и xgboost, а также совместная модель

# Загрузка датасетов с фичами и подготовка эмбедденговых данных в трейне и тесте

In [2]:
# Загрузка и подготовка эмбеддинговых данных
items = pd.read_parquet('/kaggle/input/eda-simpmod-fi-vk-2024-a01-s06-datasets/items_meta.parquet.parquet')
embeddings_df = pd.DataFrame(items['embeddings'].tolist(), index=items.index)
items = pd.concat([items, embeddings_df], axis=1)
del embeddings_df
items = items.drop(columns={'source_id',  'duration', 'embeddings'})

In [3]:
"""
links = ['/kaggle/input/feature-data/train_interactions_0_25_data_all_featues.parquet',
         '/kaggle/input/feature-data/train_interactions_25_50_data_all_featues.parquet',
         '/kaggle/input/feature-data/train_interactions_50_75_data_all_featues.parquet',
         '/kaggle/input/feature-data/train_interactions_75_100_data_all_featues.parquet',
         '/kaggle/input/feature-data/train_interactions_100_125_data_all_featues.parquet',
         '/kaggle/input/feature-data/train_interactions_125_143_data_all_featues.parquet']
"""

"\nlinks = ['/kaggle/input/feature-data/train_interactions_0_25_data_all_featues.parquet',\n         '/kaggle/input/feature-data/train_interactions_25_50_data_all_featues.parquet',\n         '/kaggle/input/feature-data/train_interactions_50_75_data_all_featues.parquet',\n         '/kaggle/input/feature-data/train_interactions_75_100_data_all_featues.parquet',\n         '/kaggle/input/feature-data/train_interactions_100_125_data_all_featues.parquet',\n         '/kaggle/input/feature-data/train_interactions_125_143_data_all_featues.parquet']\n"

In [4]:
train100 = pd.read_parquet('/kaggle/input/feature-data/train_interactions_100_125_data_all_featues.parquet')
train100 = train100.iloc[18_000_000:, :]
train125 = pd.read_parquet('/kaggle/input/feature-data/train_interactions_125_143_data_all_featues.parquet')

In [5]:
train = pd.concat([train100, train125], axis=0, ignore_index=True)
del train100, train125

In [6]:
# Добавление эмбеддинговых данных в train
train = train.merge(items, on='item_id', how='left')
train = train.drop(columns={'item_id'})

# Обучение модели CatBoost

In [7]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import pandas as pd

In [8]:
# train0_25 = pd.read_parquet('/kaggle/working/train_interactions_0_25_data_all_featues.parquet')
# train25_50 = pd.read_parquet('/kaggle/working/train_interactions_0_25_data_all_featues.parquet')
# train0_25 = train25_50.drop(columns={'user_id', 'item_id'})
# train25_50 = train25_50.drop(columns={'user_id', 'item_id'})
# train = pd.concat([train0_25, train25_50], ignore_index=True)
# del train0_25, train25_50

In [9]:
# Шаг 1: Разделяем данные
y = train["like_dislike"]
X = train.drop(columns={"like_dislike"})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
del train, X, y

In [10]:
# Категориальные признаки
# cat_features = ['gender_liked_mode', 'gender_difference']
cat_features = []

In [11]:
# Шаг 2: Создаём Pool для CatBoost
train_pool = Pool(data=X_train, label=y_train, cat_features=cat_features)
test_pool = Pool(data=X_test, label=y_test, cat_features=cat_features)

In [12]:
# Шаг 3: Создаём и обучаем модель
model = CatBoostClassifier(
    thread_count=-1,
    iterations=3000,
    depth=7,
    learning_rate=0.07,
    l2_leaf_reg=15,
    loss_function='MultiClass',   # Многоклассовая задача
    eval_metric='AUC',            # Метрика для многоклассовой задачи
    task_type='GPU',
    devices='0-1',
    verbose=100,
    early_stopping_rounds=100,
    bootstrap_type='Poisson',     # Указываем тип бутстрэпа
    subsample=0.7,                # Применяем subsample с типом бутстрэпа Poisson
    max_ctr_complexity=1,         # Длина перебора комбинаций фичей
    border_count=254,             # Устанавливаем нужное количество границ
    random_seed=42                # Фиксируем random seed
)

# Обучение модели на данных
model.fit(
    train_pool,
    eval_set=test_pool,
    use_best_model=True
)

Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.7566677	best: 0.7566677 (0)	total: 1.03s	remaining: 51m 40s
100:	test: 0.9400639	best: 0.9400639 (100)	total: 1m 6s	remaining: 31m 39s
200:	test: 0.9621929	best: 0.9622432 (194)	total: 2m 10s	remaining: 30m 12s
300:	test: 0.9649983	best: 0.9649983 (300)	total: 3m 13s	remaining: 28m 52s
400:	test: 0.9660671	best: 0.9660774 (395)	total: 4m 15s	remaining: 27m 38s
500:	test: 0.9671639	best: 0.9671742 (491)	total: 5m 18s	remaining: 26m 26s
600:	test: 0.9676710	best: 0.9676888 (551)	total: 6m 20s	remaining: 25m 18s
700:	test: 0.9681852	best: 0.9681867 (699)	total: 7m 22s	remaining: 24m 10s
800:	test: 0.9683598	best: 0.9683707 (777)	total: 8m 23s	remaining: 23m 2s
900:	test: 0.9685777	best: 0.9685777 (900)	total: 9m 24s	remaining: 21m 55s
1000:	test: 0.9685903	best: 0.9686024 (978)	total: 10m 26s	remaining: 20m 50s
1100:	test: 0.9688372	best: 0.9688372 (1100)	total: 11m 27s	remaining: 19m 45s
1200:	test: 0.9692804	best: 0.9692848 (1189)	total: 12m 27s	remaining: 18m 40s
1300:	test:

In [13]:
# Шаг 4: Оцениваем модель
# Получаем вероятности для каждого класса
y_pred = model.predict_proba(test_pool)

# Для многоклассовой классификации используем 'ovr' (one-vs-rest) или 'ovo' (one-vs-one)
roc_auc = roc_auc_score(y_test, y_pred, multi_class='ovr', average='macro')

print(f"ROC-AUC: {roc_auc:.4f}")

ROC-AUC: 0.9457


In [14]:
# Шаг 5: Создание Pool для тестовых данных
test = pd.read_parquet('/kaggle/input/feature-data/test_0_20_featues.parquet')
test = test.merge(items, on='item_id', how='left')
test = test.drop(columns={'item_id'})
test_pool = Pool(data=test, cat_features=cat_features)

In [15]:
# Шаг 6: Предсказание вероятностей
test['predicted_prob'] = model.predict_proba(test_pool)[:, 1]

In [16]:
test.iloc[:,-1:]

,predicted_prob
0,0.100804
1,0.170159
2,0.170517
3,0.100435
4,0.030966
...,...
1655115,0.000894
1655116,0.009871
1655117,0.001589
1655118,0.001497


In [17]:
# Шаг 7: Перезапись и сохранение результатов
subm = pd.read_csv('/kaggle/input/eda-simpmod-fi-vk-2024-a01-s06-datasets/sample_submission.csv')
subm['predict'] = test['predicted_prob']
subm.to_csv('/kaggle/working/submission_catboost_probabilities_18.12.csv', index=False)
subm.head()

,user_id,item_id,predict
0,1,7363,0.100804
1,1,73770,0.170159
2,1,75700,0.170517
3,1,81204,0.100435
4,1,110249,0.030966


In [18]:
# Модель с эмбеддингами позволила получить метрику 0,6444736334 на приватном датасете

In [19]:
"""
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt

X_test = X_test.sample(200000, random_state=42)
y_test = y_test.sample(200000, random_state=42)
perm_importance = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=42)
sorted_idx = perm_importance.importances_mean.argsort()
fig = plt.figure(figsize=(12, 9))
plt.barh(range(len(sorted_idx)), perm_importance.importances_mean[sorted_idx], align='center')
plt.yticks(range(len(sorted_idx)), np.array(X_test.columns)[sorted_idx])
plt.title('Permutation Importance');
"""

"\nfrom sklearn.inspection import permutation_importance\nimport matplotlib.pyplot as plt\n\nX_test = X_test.sample(200000, random_state=42)\ny_test = y_test.sample(200000, random_state=42)\nperm_importance = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=42)\nsorted_idx = perm_importance.importances_mean.argsort()\nfig = plt.figure(figsize=(12, 9))\nplt.barh(range(len(sorted_idx)), perm_importance.importances_mean[sorted_idx], align='center')\nplt.yticks(range(len(sorted_idx)), np.array(X_test.columns)[sorted_idx])\nplt.title('Permutation Importance');\n"

In [20]:
del test, subm, y_pred, train_pool, test_pool, X_train, X_test, y_train, y_test

# Автоматическое прохождение по чанкам и обучение XGBoost

In [21]:
# Функция для загрузки, обработки данных и добавления эмбеддингов
def process_datasets_with_embeddings(links, items):
    """
    Обрабатывает список датасетов, добавляя к ним эмбеддинговые фичи.

    Args:
        links (list of str): Список ссылок на parquet-даты.
        items (pd.DataFrame): Датасет с эмбеддингами.

    Yields:
        pd.DataFrame: Датасет с эмбеддингами.
    """
    embeddings_df = pd.DataFrame(items['embeddings'].tolist(), index=items.index)
    items = pd.concat([items, embeddings_df], axis=1)
    items = items.drop(columns={'source_id', 'duration', 'embeddings'})
    del embeddings_df

    for link in links:
        dataset = pd.read_parquet(link)
        dataset = dataset.merge(items, on='item_id', how='left')
        # dataset = dataset.drop(columns={'item_id'})
        yield dataset

In [22]:
def train_and_evaluate_model(dataset, iteration, output_dir, train_chanks_result):
    """
    Обучает XGBoost модель на данных, тестирует, сохраняет модель и записывает метрики.

    Args:
        dataset (pd.DataFrame): Обработанный датасет.
        iteration (int): Индекс итерации для сохранения модели.
        output_dir (str): Путь для сохранения моделей.
        train_chanks_result (pd.DataFrame): Таблица результатов.

    Returns:
        pd.DataFrame: Обновлённая таблица результатов.
    """
    # Преобразуем метки в диапазон [0, num_class-1]
    dataset["like_dislike"] = dataset["like_dislike"] + 1  # Сдвигаем метки: -1 -> 0, 0 -> 1, 1 -> 2

    # Проверяем наличие сохранённых моделей
    model_files = [
        f for f in os.listdir(output_dir) if f.startswith("model_iteration_") and f.endswith(".json")
    ]
    if model_files:
        # Находим последнюю модель
        model_files.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))
        latest_model_file = model_files[-1]
        latest_model_path = os.path.join(output_dir, latest_model_file)
        print(f"Loading model from: {latest_model_path}")
        
        # Загружаем существующую модель
        base_model = xgb.Booster()
        base_model.load_model(latest_model_path)
        continue_training = True
    else:
        print("No existing model found. Creating a new model.")
        base_model = None
        continue_training = False

    # Разделяем данные
    y = dataset["like_dislike"]
    X = dataset.drop(columns=["like_dislike"])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
    del dataset, X, y

    # Создаём DMatrix для XGBoost
    train_dmatrix = xgb.DMatrix(data=X_train, label=y_train)
    test_dmatrix = xgb.DMatrix(data=X_test, label=y_test)

    # Настраиваем параметры
    params = {
        "objective": "multi:softprob",
        "eval_metric": "mlogloss",
        "eta": 0.07,
        "max_depth": 8,
        "subsample": 0.7,
        "colsample_bytree": 0.7,
        "lambda": 25,  # Увеличиваем L2-регуляризацию
        "alpha": 15,   # Добавляем L1-регуляризацию
        "num_class": len(set(y_train)),  # Количество уникальных классов
        "random_state": 42,
        "tree_method": "hist",  # Используем метод hist для CPU, либо device для GPU
        "device": "cuda"  # Если используете GPU, указать "cuda"
    }

    # Если продолжаем обучение, устанавливаем checkpoint
    if continue_training:
        print("Continuing training from the previous model.")
        base_iteration = int(latest_model_file.split('_')[-1].split('.')[0]) + 1
    else:
        base_iteration = 0
    
    # Обучаем модель
    evals = [(train_dmatrix, "train"), (test_dmatrix, "validation")]
    model = xgb.train(
        params=params,
        dtrain=train_dmatrix,
        num_boost_round=250,  # Количество новых раундов обучения
        evals=evals,
        xgb_model=base_model,  # Указываем модель для продолжения обучения
        verbose_eval=10  # Выводить информацию каждые 10 итераций
    )
    
    # Оцениваем метрику ROC-AUC
    y_pred = model.predict(test_dmatrix)
    y_pred_classes = y_pred.argmax(axis=1)  # Преобразуем вероятности в классы
    y_pred_original = y_pred_classes - 1  # Возвращаем метки в исходный диапазон (-1, 0, 1)
    roc_auc = roc_auc_score(y_test - 1, y_pred, multi_class='ovr', average='macro')  # Обратный сдвиг для сравнения
    print(f"Iteration {iteration} - ROC-AUC: {roc_auc:.4f}")
    
    # Сохраняем модель
    model_path = os.path.join(output_dir, f"model_iteration_{iteration}.json")
    model.save_model(model_path)
    
    # Записываем результаты
    new_result = pd.DataFrame({"iteration": [iteration + base_iteration], "roc_auc": [roc_auc]})
    train_chanks_result = pd.concat([train_chanks_result, new_result], ignore_index=True)
    return train_chanks_result


In [23]:
# Запуск обучения
if __name__ == "__main__":
    items = pd.read_parquet('/kaggle/input/eda-simpmod-fi-vk-2024-a01-s06-datasets/items_meta.parquet.parquet')
    links = [
        '/kaggle/input/feature-data/train_interactions_0_25_data_all_featues.parquet',
        '/kaggle/input/feature-data/train_interactions_25_50_data_all_featues.parquet',
        '/kaggle/input/feature-data/train_interactions_50_75_data_all_featues.parquet',
        '/kaggle/input/feature-data/train_interactions_75_100_data_all_featues.parquet',
        '/kaggle/input/feature-data/train_interactions_100_125_data_all_featues.parquet',
        '/kaggle/input/feature-data/train_interactions_125_143_data_all_featues.parquet'
    ]

    output_dir = "/kaggle/working"
    train_chanks_result = pd.DataFrame(columns=["iteration", "bestTest", "roc_auc"])

    # Проходим по датасетам и обучаем модель
    for i, dataset in enumerate(process_datasets_with_embeddings(links, items)):
        train_chanks_result = train_and_evaluate_model(dataset, i, output_dir, train_chanks_result)

    # Сохраняем результаты
    train_chanks_result.to_csv(os.path.join(output_dir, "train_chanks_result.csv"), index=False)

No existing model found. Creating a new model.
[0]	train-mlogloss:1.00809	validation-mlogloss:1.00811
[10]	train-mlogloss:0.49910	validation-mlogloss:0.49921
[20]	train-mlogloss:0.29816	validation-mlogloss:0.29833
[30]	train-mlogloss:0.20942	validation-mlogloss:0.20964
[40]	train-mlogloss:0.16727	validation-mlogloss:0.16754
[50]	train-mlogloss:0.14677	validation-mlogloss:0.14709
[60]	train-mlogloss:0.13572	validation-mlogloss:0.13608
[70]	train-mlogloss:0.12942	validation-mlogloss:0.12982
[80]	train-mlogloss:0.12606	validation-mlogloss:0.12649
[90]	train-mlogloss:0.12391	validation-mlogloss:0.12438
[100]	train-mlogloss:0.12257	validation-mlogloss:0.12307
[110]	train-mlogloss:0.12169	validation-mlogloss:0.12222
[120]	train-mlogloss:0.12103	validation-mlogloss:0.12159
[130]	train-mlogloss:0.12058	validation-mlogloss:0.12117
[140]	train-mlogloss:0.12025	validation-mlogloss:0.12086
[150]	train-mlogloss:0.12000	validation-mlogloss:0.12065
[160]	train-mlogloss:0.11980	validation-mlogloss:0.1

/tmp/ipykernel_23/1599133232.py:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  train_chanks_result = pd.concat([train_chanks_result, new_result], ignore_index=True)


Loading model from: /kaggle/working/model_iteration_0.json
Continuing training from the previous model.
[0]	train-mlogloss:0.11972	validation-mlogloss:0.11974
[10]	train-mlogloss:0.11950	validation-mlogloss:0.11957
[20]	train-mlogloss:0.11938	validation-mlogloss:0.11948
[30]	train-mlogloss:0.11927	validation-mlogloss:0.11942
[40]	train-mlogloss:0.11918	validation-mlogloss:0.11937
[50]	train-mlogloss:0.11911	validation-mlogloss:0.11933
[60]	train-mlogloss:0.11904	validation-mlogloss:0.11930
[70]	train-mlogloss:0.11898	validation-mlogloss:0.11927
[80]	train-mlogloss:0.11892	validation-mlogloss:0.11925
[90]	train-mlogloss:0.11886	validation-mlogloss:0.11923
[100]	train-mlogloss:0.11880	validation-mlogloss:0.11920
[110]	train-mlogloss:0.11874	validation-mlogloss:0.11918
[120]	train-mlogloss:0.11869	validation-mlogloss:0.11916
[130]	train-mlogloss:0.11863	validation-mlogloss:0.11914
[140]	train-mlogloss:0.11857	validation-mlogloss:0.11912
[150]	train-mlogloss:0.11852	validation-mlogloss:0.1

In [24]:
# Предсказание вероятностей
# Шаг 1: Загрузка последней модели
output_dir = '/kaggle/working'  # Путь к папке с моделями
model_files = [
    f for f in os.listdir(output_dir) if f.startswith("model_iteration_") and f.endswith(".json")
]

if model_files:
    # Находим последнюю модель
    model_files.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))
    latest_model_file = model_files[-1]
    latest_model_path = os.path.join(output_dir, latest_model_file)
    print(f"Loading model from: {latest_model_path}")
    
    # Загружаем модель
    model = xgb.Booster()
    model.load_model(latest_model_path)
else:
    raise ValueError("No model found to load")

# Шаг 2: Загрузка тестовых данных
items = pd.read_parquet('/kaggle/input/eda-simpmod-fi-vk-2024-a01-s06-datasets/items_meta.parquet.parquet')
embeddings_df = pd.DataFrame(items['embeddings'].tolist(), index=items.index)
items = pd.concat([items, embeddings_df], axis=1)
del embeddings_df
items = items.drop(columns={'source_id',  'duration', 'embeddings'})

test = pd.read_parquet('/kaggle/input/feature-data/test_0_20_featues.parquet')
test = test.merge(items, on='item_id', how='left')
# test = test.drop(columns={'item_id'})

# Шаг 3: Обработка столбцов с типом 'object'
# test = test.apply(pd.to_numeric, errors='ignore')  # Преобразование в числовые
# test = test.select_dtypes(include=[int, float, bool, 'category'])

# Шаг 4: Подготовка тестовых данных в формат DMatrix
test_dmatrix = xgb.DMatrix(test)

# Шаг 5: Получение вероятностей для каждого класса
y_pred = model.predict(test_dmatrix)  # Это будет массив с вероятностями для каждого класса

# Шаг 6: Создание файла для submission
subm = pd.read_csv('/kaggle/input/eda-simpmod-fi-vk-2024-a01-s06-datasets/sample_submission.csv')
subm['predict'] = y_pred.tolist()  # Для многоклассовой классификации, y_pred будет массивом вероятностей для каждого объекта

# Для многоклассовой классификации, y_pred будет 2D массивом:
# Количество строк — это количество объектов, количество столбцов — количество классов.
# Если нужно для каждого объекта предсказать вероятность определенного класса, 
# можно использовать `y_pred[:, class_index]`, например для класса 0:
# subm['predict'] = y_pred[:, 0]  # Предсказание вероятности для класса 0

# Шаг 7: Вычисление predict на основании полученного многомерного массива
subm_predict = pd.DataFrame(subm['predict'].tolist(), index=subm.index)
subm = pd.concat([subm, subm_predict], axis=1)
subm['predict'] = subm[2] - subm[0]
subm = subm[['user_id', 'item_id', 'predict']]

# Clip data for 0 (makes trash results)
# subm['predict'] = subm['predict'].apply(lambda x: max(x, 0))

# Сохраняем результат
subm.to_csv('/kaggle/working/submission_xgb_probabilities.csv', index=False)
subm.head()

Loading model from: /kaggle/working/model_iteration_5.json


,user_id,item_id,predict
0,1,7363,0.125653
1,1,73770,0.142450
2,1,75700,0.180228
3,1,81204,0.126991
4,1,110249,0.045290


In [25]:
subm.describe()

,user_id,item_id,predict
count,1.655120e+06,1.655120e+06,1.655120e+06
mean,9.434374e+04,1.703528e+05,1.087732e-01
std,5.249043e+04,9.780802e+04,1.447175e-01
min,1.000000e+00,1.000000e+00,-9.003843e-01
25%,4.955375e+04,8.476400e+04,6.855862e-03
50%,9.568500e+04,1.714620e+05,4.994055e-02
75%,1.398602e+05,2.527330e+05,1.546676e-01
max,1.834030e+05,3.377250e+05,9.767173e-01
